In [1]:
import pandas as pd

In [2]:
meta = pd.read_excel('../.data_sources/New/GSE198449/GSE198449_PCR_and_Symptoms_Full_Info.xlsx', header=None)

In [3]:
meta = meta.iloc[22:, :]

In [4]:
meta.index = meta.iloc[:, 1]

In [5]:
meta = meta.drop(columns=1)

In [6]:
meta = meta.dropna(subset=[4])

In [7]:
meta.columns = ['enrollment batch', 'participant id', 'collection time', 'cov2', 'symptom']

In [9]:
infected_ids = set(meta.loc[meta['cov2'] == 'Detected']['participant id'])

In [14]:
samples = dict()
for id0 in infected_ids:
    detected_samples = meta.loc[(meta['participant id'] == id0) & (meta['cov2'] == 'Detected')]
    detected_time0 = min(detected_samples['collection time'])
    detected_id = detected_samples.loc[detected_samples['collection time'] == detected_time0].index[0]
    control_samples = meta.loc[(meta['participant id'] == id0) & (meta['cov2'] == 'Not') & (meta['collection time'] < detected_time0)]
    if control_samples.empty: continue
    control_time_before_infection = max(control_samples['collection time'])
    control_id = control_samples.loc[control_samples['collection time'] == control_time_before_infection].index[0]
    samples[id0] = {"control":control_id, "infected":detected_id}

In [16]:
read_counts = pd.read_csv('../.data_sources/New/GSE198449/GSE198449_featureCounts.txt', sep='\t')
read_counts

,Geneid,20_5063-T00_P1,20_5134-T00_P1,20_5218-T00_P1,20_5246-T00_P1,20_5326-T00_P1,20_5342-T00_P1,20_5411-T00_P1,20_5420-T00_P1,20_5513-T00_P1,...,20_8860-T00_P27,20_8991-T00_P27,20_9127-T00_P27,20_9127-T91_P27,20_9153-T00_P27,20_9153-T63_P27,20_9520-T00_P27,20_9719-T00_P27,20_9719-T91_P27,20_9749-T00_P27
0,ENSG00000000003.14,5.00,2.00,8.00,0.00,1.00,5.00,2.00,3.00,2.0,...,6.0,4.0,1.00,7.00,3.00,3.00,4.00,11.00,1.00,14.00
1,ENSG00000000005.6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,...,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,ENSG00000000419.12,282.00,205.00,237.00,254.00,211.00,222.00,336.00,357.00,184.0,...,518.0,178.0,250.00,330.00,386.00,259.00,364.00,341.00,407.00,447.00
3,ENSG00000000457.14,393.42,235.43,378.70,325.73,356.84,294.05,347.48,485.67,241.1,...,619.3,139.1,448.82,581.35,376.11,341.86,388.45,565.39,626.18,562.39
4,ENSG00000000460.17,35.58,35.57,40.30,39.27,30.16,25.95,44.52,46.33,25.9,...,71.7,27.9,75.18,77.65,105.89,64.14,27.55,97.61,74.82,59.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58924,ENSG00000286268.1,0.00,1.98,1.83,4.08,1.76,8.95,0.00,0.00,0.0,...,0.0,0.0,2.11,1.89,0.00,8.23,0.00,0.00,0.00,4.80
58925,ENSG00000286269.1,1.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.0,...,0.0,0.0,0.00,0.00,0.00,0.00,0.00,1.00,1.00,0.00
58926,ENSG00000286270.1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,...,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
58927,ENSG00000286271.1,0.00,1.00,0.00,5.00,0.00,0.00,0.00,0.00,2.0,...,2.0,0.0,0.00,5.00,0.00,0.00,0.00,5.00,5.00,5.00


In [32]:
sample_ids = [i for x in samples.items() for i in (x[1]['control'], x[1]['infected'])]

In [54]:
read_counts.columns = read_counts.columns.str.replace(r'_P\d*', '')

/tmp/ipykernel_545/2008950950.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  read_counts.columns = read_counts.columns.str.replace(r'_P\d*', '')


In [61]:
read_counts[sample_ids[0]]

KeyError: '20_6132-T28'